# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686580


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:06<01:24,  3.01s/it]

Rendering models:  10%|█         | 3/30 [00:06<00:58,  2.16s/it]

Rendering models:  17%|█▋        | 5/30 [00:06<00:38,  1.53s/it]

Rendering models:  23%|██▎       | 7/30 [00:06<00:25,  1.10s/it]

...

Rendering models:  80%|████████  | 24/30 [00:07<00:00,  6.33it/s]

Rendering models:  87%|████████▋ | 26/30 [00:08<00:00,  6.88it/s]

Rendering models:  93%|█████████▎| 28/30 [00:08<00:00,  8.17it/s]

Rendering models: 100%|██████████| 30/30 [00:08<00:00,  9.13it/s]

firefighterwolf13                     0.002615
smithkurtis                           0.000199
richiemac                             0.000133
Camelron                              0.000191
Staceyann116                          0.015025
asucich                               0.000333
djswanso                              0.018828
kjkavene                              0.000188
not-logged-in-ff40b895d06514783cbc    0.002834
bldelacr                              0.004241
ncaro                                 0.001194
pollux2728                            0.007493
not-logged-in-388222ff3e4deda2c904    0.074718
coyotes54                             0.000136
Dylano                                0.000810
not-logged-in-dddbb97cfd84354416f5    0.000475
pangeli5                              0.000288
RaulE                                 0.000587
not-logged-in-447d0673a41f48ce6d98    0.000960
acapirala                             0.000631
Lavadude                              0.005210
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())